<h1><center>Big Data Algorithms Techniques & Platforms</center></h1>
<h2>
<hr style=" border:none; height:3px;">
<center>Lab 3: Advanced Spark</center>
<hr style=" border:none; height:3px;">
</h2>

# Introduction


<p align="justify">
<font size="3">
In this set of exercises you will continue using Spark for implementing algorithms that apply computations on documents and matrices.
You will be required to reason on tables, their representation, and arriving implementing the matrix multiplication, and finally the pagerank algorithm.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the following cell in order to initialize the _SparkContext_.**</font>
<hr style=" border:none; height:2px;">
</p>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar zxvf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local")
sc = SparkContext(conf = conf)
print("initialization successful!")

import numpy as np
import random as rn

seed_value=0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

<p align="justify">
<font size="3">
Upload the folder data.zip inside the colab data folder and then execute the following code.
</font>
</p>

In [ ]:
!apt-get install unzip
!unzip data.zip

# A. Matrix Representation

<p align="justify">
<font size="3">
**Please read carefully this section that presents how matrices will be represented in this assignment.**
</font>
</p>

<p align="justify">
<font size="3">
Our input matrices are stored 
in textual files
As an example, the file matrix-a.txt_ looks like as follows:
<p>
0 1 2 4<br>
1 2 3 10<br>
2 12 15 150<br>
</p>
</font>
</p>
<p>
<font size="3">
Each line is a row in a matrix $A$. The first number of the line is the 
row identifier (starting from 0), the subsequent values (separated by a whitespace)
are the elements in each column of the row. The matrix represented in this file is the 
following:
<p>
<center>
  $A= \begin{bmatrix}
    1 & 2 & 4   \\
    2 & 3 & 10  \\
    12 & 15 & 150
\end{bmatrix}$
</center>
</font>
</p>


<p>
<font size="3">
We provide the implementation of  basic functions to load a matrix from file, visualize it
and get attributes.
</font>
</p>

## A.1 Function $loadMatrix$

<p>
<font size="3">
The function $loadMatrix()$ loads a matrix from a file.
It takes in the name of the file and returns an RDD containing the matrix.

Each element of an RDD matrix is a key-value pair, where the key is the coordinate (row identifier, column identifier) of an element, and the value is the element itself.
For instance, the RDD corresponding to the matrix $A$ is the following:
<p>
$( (0, 0), 1 ), ( (0, 1), 2 ), ( (0, 2), 4 ), ( (1, 0), 2 ), ( (1, 1), 3 ), ( (1, 2), 10 ), ( (2, 0), 12 ), ( (2, 1), 15 ), ( (2, 2), 150 ) $
</p>
</font>
</p>

## A.2 Function $shape$

<p>
<font size="3">
The function $shape()$ takes in an RDD matrix and returns the size of the matrix as a pair $(nbRows, nbCols)$, where $nbRows$ (resp., $nbCols$) denotes the number of rows (resp., columns) of the matrix.
</font>
</p>

## A.3 Function $collect$

<p>
<font size="3">
The function $collect()$ takes in an RDD matrix and returns a representation of the matrix as a Python list $L$. Each element of $L$ is itself a list that corresponds to a row in the matrix.
For instance, the output of the function $collect$ for the matrix $A$ is as follows:   

<p>

$[ [1, 2, 4], [2, 3, 10], [12, 15, 150] ]$

</p>

</font>
</p>


## A.4 Function $nice$

<p>
<font size="3">
The function $nice()$ prints the matrix in a nice and readable way.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the following cell in order to initialize the definition of the functions**</font>
<hr style=" border:none; height:2px;">
</p>

In [3]:
'''
Loads a matrix from a file.
Takes in: the name of the input file
Returns: an RDD containing the matrix
'''
def loadMatrix(filename):
    # Load the file into an RDD matrix
    matrix = sc.textFile(filename)
    # Splits each line. Each element is a list [nbRow, e1, e2, ..., ej]
    matrix = matrix.map(lambda line : line.split(' '))
    # Convert each element to a number (the first is an integer, the others are float)
    matrix = matrix.map(lambda row: [int(row[0])] + [float(row[i]) for i in range(1, len(row))])
    # Get an RDD where each element is a key-value pair ((row, col), element)
    matrix = matrix.flatMap(lambda row: [((row[0], j-1), row[j]) for j in range(1, len(row))])
    return matrix

'''
Returns the number of rows and colums of the matrix
Takes in: An RDD representing a matrix
Returns: the size of the matrix as (nbRows, nbCols)
'''
def shape(matrix):
    M = collect(matrix)
    if len(M) == 0:
        return (0, 0)
    else:
        return (len(M), len(M[0]))

'''
Returns a matrix represented as a list of lists.
Takes in: an RDD representing a matrix
Returns: the matrix represented as a list of lists.
'''
def collect(matrix):
    # Obtain an RDD, where the key is the row identifier and the value is (colId, element)
    matrix = matrix.map(lambda x: (x[0][0], (x[0][1], x[1])))
    # Groups all the values in a row.
    matrix = matrix.groupByKey()
    # Sorts the element by row identifier.
    matrix = matrix.sortByKey()
    # Sort the elements by column identifier.
    matrix = matrix.map(lambda x: sorted(list(x[1])))
    # Now obtain an RDD, where each element is a list containing the elements of a row.
    matrix = matrix.map(lambda row: [x[1] for x in row])
    # Finally, return the RDD as a Python list.
    return matrix.collect()
    
'''
Prints the matrix in a nice way.
Takes in: the name of the matrix (var) and the matrix in the form of an RDD.
'''
def nice(var, matrix):
    # Obtain a representation of the matrix as a Python list.
    M = collect(matrix)
    # Print the name of the matrix
    print("Matrix ", var)
    # Print the matrix and format the output nicely
    print('\n'.join([''.join(['{:12.2f}'.format(item) for item in row]) 
      for row in M]))


## Exercise 1. Matrix Addition

<p align="justify">
<font size="3">
The code below loads two matrices $A$ and $B$ from file and calls the function $sum()$ to compute $A+B$.
</font>
</p>

<p>
<font size="3">
The function $sum()$ takes in:
<ul>
<li> $A$: an RDD containing the first matrix.
<li> $B$: an RDD containing the second matrix.
</ul>
The function $sum()$ returns an RDD containing the matrix obtained by summing $A$ and $B$.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the function $sum()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [4]:
'''
Computes the sum of two matrices.
Takes in: two RDDs containing the input matrices
Returns: the RDD containing the sum of the two input matrices
'''


def sum(A, B):
    
    ################## COMPLETE HERE FOLLOWING THE INSTRUCTIONS ##################
    
    # Each element of the RDD A and B is ((r,c), e), where e is an element of the matrix and (r, c) is the 
    # coordinate of the element in terms of row and column.
    
    # 1. Put the two RDDs A and B together. Use the transformation union. Remember that a transformation 
    # always returns a new RDD with the result of the transformation.
    C = A.union(B)
    
    #2. Transforms the RDD C into one where the values having the same key (i.e., same row and column) 
    # are summed together. Which transformation are you going to use on C?
    C = C.reduceByKey(lambda x, y : x + y) 
    # We return the RDD containing the sum of the two input matrices
    return C

    ################## END OF MODIFICATIONS ##################
        


# Load matrix A from file and print it.
A = loadMatrix("./data/matrix-a.txt")
nice("A", A)

# Load matrix B from file and print it.
B = loadMatrix("./data/matrix-b.txt")
nice("B", B)

# Compute A+B and print it
C = sum(A, B)
nice("C", C)

############################################################## 
#YOU SHOULD OBTAIN THE FOLLOWING MATRIX C AS RESULT
# 5.00        4.00        6.00      324.00       23.00
# 3.00        6.00       13.00      333.00      423.00
# 35.00       49.00      162.00       12.00        0.00
##############################################################


Matrix  A
        1.00        2.00        4.00
        2.00        3.00       10.00
       12.00       15.00      150.00
Matrix  B
        4.00        2.00        2.00      324.00       23.00
        1.00        3.00        3.00      333.00      423.00
       23.00       34.00       12.00       12.00        0.00
Matrix  C
        5.00        4.00        6.00      324.00       23.00
        3.00        6.00       13.00      333.00      423.00
       35.00       49.00      162.00       12.00        0.00


## Exercise 2. Scalar Multiplication

<p>
<font size="3">
The code below calls the function $scalarMultiply()$ to obtain the matrix $c\times A$, where $c$ is a scalar value.    
</font>
</p>

<p>
<font size="3">
The function $scalarMultiply()$ takes in:
<ul>
<li> $c$: a scalar value.
<li> $M$: an RDD containing a matrix.
</ul>
The function $scalarMultiply()$ returns an RDD containing the matrix obtained by multiplying $c$ with the input matrix.
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the function $scalarMultiply()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [5]:
'''
Computes the scalar multiplication.
Takes in a scalar value c and an RDD matrix M
Returns the RDD containing the matrix resulting from the scalar multiplication c * M.
'''
def scalarMultiply(c, M):
    # Apply a transformation on M, so each element of the matrix M is multiplied by c
    return M.map(lambda el: ((el[0][0], el[0][1]), c*el[1]))



# Prints 
nice("A", A)
nice("2*A", scalarMultiply(2, A))

############################################################## 
# THE RESULT SHOULD BE 
#2.00        4.00        8.00
#4.00        6.00       20.00
#24.00       30.00      300.00
##############################################################

Matrix  A
        1.00        2.00        4.00
        2.00        3.00       10.00
       12.00       15.00      150.00
Matrix  2*A
        2.00        4.00        8.00
        4.00        6.00       20.00
       24.00       30.00      300.00


## Exercise 3. Matrix Multiplication


<p>
<font size="3">
We want to implement a function $multiply()$ to obtain the matrix $A \times B$.
</font>
</p>

<p>
<font size="3">
The function $multiply()$ takes in:
<ul>
<li> $A$: an RDD containing the first matrix.
<li> $B$: an RDD containing the second matrix.
</ul>
The function $multiply$ returns an RDD containing the matrix obtained by multiplying the first and the second matrix.
The multiplication can only be computed if the number of columns of $A$ equals the number of rows of $B$.
</font>
</p>


<p>
<font size="3">
Let $A$ be an $n \times m$ matrix and $B$ an $m \times p$ matrix.
The matrix $C = A \times B$ is a $n \times p$ matrix, where each element $c_{i, k}$ is computed as 
follows:
<center>
  $c_{i, k} = \sum\limits_{j=0}^{m-1} a_{i, j} \cdot b_{j, k} \quad\quad (1)$ 
</center>
</font>
</p>


<p>
<font size="3">
One possible implementation of this function is based on a MapReduce schema.
Remember that in a MapReduce schema, the idea is to group elements by a key and apply a function to the elements 
that share the same key.
As you can see, for a given $(i, k)$, the element of $A$ that is in the j-th column is multiplied by the value of $B$ that is in the j-th row, for any column $j$.
Therefore, we can change the representation of $A$ and $B$ so that their elements are indexed by using $j$ as the key.
</font>
</p>

<p>
<font size="3">
More specifically, we can represent the matrix $A$ as follows:
<center>    
    $(j, (0, i, a_{i, j})) \quad 0 \leq i \leq n-1 \quad 0 \leq j \leq m-1  \quad\quad (2)$
</center>    
where the value $0$ in the triple $(0, i, a_{i, j})$ means that the element $a_{i, j}$ comes from the matrix $A$.
</font>
</p>


<p>
<font size="3">
Similarly, we can represent the matrix $B$ as follows:
<center>    
    $(j, (1, k, b_{j, k})) \quad 0 \leq j \leq m-1 \quad 0 \leq k \leq p-1 \quad\quad (3) $
</center>    
where the value $1$ in the triple $(1, k, b_{j, k})$ means that the element $b_{j, k}$ comes from the matrix $B$.
</font>
</p>

<p>
<font size="3">
As a first step, we want to code two functions $transformA()$ and $transformB()$ to obtain the two representations of $A$ and $B$ respectively, as described in Equation (2) and (3).
The two representations returned by both functions **must be RDDs**.
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the functions $transformA()$ and $transformB()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [6]:
################## COMPLETE HERE FOLLOWING THE INSTRUCTIONS ##################

'''
Transforms the RDD matrix A into an RDD as described in Equation (2)
'''
def transformA(A):
    return A.map(lambda el: (el[0][1], (0, el[0][0], el[1])))
'''
Transforms the RDD matrix B into an RDD as described in Equation (3) '''
def transformB(B):
    return B.map(lambda el: (el[0][0], (1, el[0][1], el[1])))


# Displayes the two matrices
nice("A", A)
nice("B", B)

# Transforms them
Atransformed = transformA(A)
Btransformed = transformB(B)

# Display the result.
print("\n********** Representation for A ************\n")
print(Atransformed.collect())
print("\n********** Representation for B ************\n")
print(Btransformed.collect())


Matrix  A
        1.00        2.00        4.00
        2.00        3.00       10.00
       12.00       15.00      150.00
Matrix  B
        4.00        2.00        2.00      324.00       23.00
        1.00        3.00        3.00      333.00      423.00
       23.00       34.00       12.00       12.00        0.00

********** Representation for A ************

[(0, (0, 0, 1.0)), (1, (0, 0, 2.0)), (2, (0, 0, 4.0)), (0, (0, 1, 2.0)), (1, (0, 1, 3.0)), (2, (0, 1, 10.0)), (0, (0, 2, 12.0)), (1, (0, 2, 15.0)), (2, (0, 2, 150.0))]

********** Representation for B ************

[(0, (1, 0, 4.0)), (0, (1, 1, 2.0)), (0, (1, 2, 2.0)), (0, (1, 3, 324.0)), (0, (1, 4, 23.0)), (1, (1, 0, 1.0)), (1, (1, 1, 3.0)), (1, (1, 2, 3.0)), (1, (1, 3, 333.0)), (1, (1, 4, 423.0)), (2, (1, 0, 23.0)), (2, (1, 1, 34.0)), (2, (1, 2, 12.0)), (2, (1, 3, 12.0)), (2, (1, 4, 0.0))]


<p>
<font size="3">
In order to group all the elements of both matrices by the key $j$, we need to merge the two RDDs $Atransformed$ and $Btransformed$.
The function $merge()$ declared below takes in $Atransformed$ and $Btransformed$ and returns an RDD that results from the union of the two input RDDs.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the functions $merge()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [7]:
'''
Returns the union of Atransformed and Btransformed.
'''
def merge(Atransformed, Btransformed):    
    # Put together the two input RDDs
    R = Atransformed.union(Btransformed) 
    return R


nice("A", A)
nice("B", B)
    
merged = merge(Atransformed, Btransformed)    

print("\n********** Representation for merged ************\n")
print(merged.collect())

######################################################################
# Note that in the output, each element is (j, L), where
# L is a list that contains all the elements in the j-th column of A
# and all the elements in j-th row of B
######################################################################

Matrix  A
        1.00        2.00        4.00
        2.00        3.00       10.00
       12.00       15.00      150.00
Matrix  B
        4.00        2.00        2.00      324.00       23.00
        1.00        3.00        3.00      333.00      423.00
       23.00       34.00       12.00       12.00        0.00

********** Representation for merged ************

[(0, (0, 0, 1.0)), (1, (0, 0, 2.0)), (2, (0, 0, 4.0)), (0, (0, 1, 2.0)), (1, (0, 1, 3.0)), (2, (0, 1, 10.0)), (0, (0, 2, 12.0)), (1, (0, 2, 15.0)), (2, (0, 2, 150.0)), (0, (1, 0, 4.0)), (0, (1, 1, 2.0)), (0, (1, 2, 2.0)), (0, (1, 3, 324.0)), (0, (1, 4, 23.0)), (1, (1, 0, 1.0)), (1, (1, 1, 3.0)), (1, (1, 2, 3.0)), (1, (1, 3, 333.0)), (1, (1, 4, 423.0)), (2, (1, 0, 23.0)), (2, (1, 1, 34.0)), (2, (1, 2, 12.0)), (2, (1, 3, 12.0)), (2, (1, 4, 0.0))]


<p>
<font size="3">
Now we can group the values of the RDD $merged$ obtained above by their key $j$. 
We define a function $group()$ that returns an RDD obtained by grouping the values of the input RDD by their key.
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the functions $group()$ and execute the code**</font>
<hr style=" border:none; height:2px;">
</p>



In [8]:
'''
Returns an RDD where the values of the input RDD are grouped by their key.
'''
def group(merged):
      # Groups the element of the input RDD by key.
    R = merged.groupByKey() 
    return R

nice("A", A)
nice("B", B)


nice("A", A)
nice("B", B)
    
grouped = group(merged)    

print("\n********** Representation for grouped ************\n")
L = grouped.collect()
print('[')
for l in L:
    print("(",l[0], ",", end='', sep='')
    print("[", end='')
    for el in l[1]:
        print(el, end="")
    print("],")
print(']')

######################################################################
# Note that in the output, each element is (j, L), where
# L is a list that contains all the elements in the j-th column of A
# and all the elements in j-th row of B
######################################################################

Matrix  A
        1.00        2.00        4.00
        2.00        3.00       10.00
       12.00       15.00      150.00
Matrix  B
        4.00        2.00        2.00      324.00       23.00
        1.00        3.00        3.00      333.00      423.00
       23.00       34.00       12.00       12.00        0.00
Matrix  A
        1.00        2.00        4.00
        2.00        3.00       10.00
       12.00       15.00      150.00
Matrix  B
        4.00        2.00        2.00      324.00       23.00
        1.00        3.00        3.00      333.00      423.00
       23.00       34.00       12.00       12.00        0.00

********** Representation for grouped ************

[
(0,[(0, 0, 1.0)(0, 1, 2.0)(0, 2, 12.0)(1, 0, 4.0)(1, 1, 2.0)(1, 2, 2.0)(1, 3, 324.0)(1, 4, 23.0)],
(2,[(0, 0, 4.0)(0, 1, 10.0)(0, 2, 150.0)(1, 0, 23.0)(1, 1, 34.0)(1, 2, 12.0)(1, 3, 12.0)(1, 4, 0.0)],
(1,[(0, 0, 2.0)(0, 1, 3.0)(0, 2, 15.0)(1, 0, 1.0)(1, 1, 3.0)(1, 2, 3.0)(1, 3, 333.0)(1, 4, 423.0)],
]


<p>
<font size="3">
Each element of the RDD $grouped$ obtained above is a key-value pair, where the key is the index $j$ and the value is a list $L$ containing all the triples corresponding to the elements of matrix $A$ in the $j-$th column  and the elements of matrix $B$  in the $j-$row, as follows: 
<p>
<center>
$(0, i, a_{i, j})\ 0 \leq i \leq n-1 \quad (1, k, b_{j, k})\ 0 \leq k \leq p-1$
</center>
</p>
<p>
Remember that all triples corresponding to matrix $A$ have 0 as their first value, while those corresponding to matrix $B$ have 1.
</p>
<p>
From Equation (1), you can see that the product $a_{i, j} \cdot b_{j, k}$ contributes to the value $c_{i, k}$, for $0 \leq i \leq n-1$ and $0 \leq k \leq p-1$. 
Therefore, given the list $L$, we associate each value $a_{i, j} \cdot b_{j, k}$  to the pair $(i, k)$.
</p>
<p>
In other words, we now transform the RDD $grouped$ into an RDD where 
each element is a key-value pair, where the key is $(i, k)$ and the value is $a_{i, j} \cdot b_{j, k}$.

</p>    
<p>
In the code below, the function $multiplyElements()$ is given that takes in a value $(j, L)$ of the RDD $grouped$ 
and returns a list, where each element is a pair $((i, k), a_{i, j} \cdot b_{j, k})$, $0 \leq i \leq n-1$ and $0 \leq k \leq p-1$.  
</p>
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the function $groupProducts()$ that:
    <ul>
      <li> Takes in the RDD $grouped$.
      <li> Applies the function $multiplyElements()$ to each element of $grouped$.
      <li> Returns an RDD where each element is $((i, k), a_{i, j} \cdot b_{j, k})$, $0 \leq i \leq n-1$ and $0 \leq k \leq p-1$. 
    </ul>
    Execute the code.**</font>
<hr style=" border:none; height:2px;">
</p>




In [9]:
import sys

'''
Multiplies each element from matrix A with each element from 
matrix B in the list L (see description above).
Takes in: a value (j, L) of the RDD grouped.
Returns: a list where each element is ((i, k), a_ij * b_jk)
'''
def multiplyElements(value):
    j = value[0]
    L = value[1]
    
    '''
    The output key-value pairs.
    '''
    kv = []
    '''
    Maybe not necessary, we make sure that all triples with 
    the first element 0 (those from matrix A)
    comes before any triple from matrix B.
    '''
    L = sorted(list(L))
    
    '''
    For convenience, we store the triples from matrix A
    and those from matrix B in two separate lists 
    LA and LB.
    '''
    sep = 0
    while L[sep][0] == 0:
        sep += 1
    LA = [L[i] for i in range(0, sep)]
    LB = [L[i] for i in range(sep, len(L))]
    '''
    For each element (0, i, a_ij) in LA  
    and each element (1, k, b_jk) in LB, 
    we add the pair ((i, k), a_ij * b_jk) to kv.
    '''
    for a in LA:
        for b in LB:
            i = a[1]
            k = b[1]
            kv.append(((i, k), a[2]*b[2]))
    return kv

'''
Returns an RDD where each value is a pair ((i, k), a_ij * b_jk)
'''
def groupProducts(grouped):
    # Apply a transformation to the input RDD to get an RDD as described in the text.
    # Which tranformations are you going to use? map or flatMap?
    return grouped.flatMap(multiplyElements) 
nice("A", A)
nice("B", B)

print("\n********** Representation for multipliedElements ************\n")
multipliedElements = groupProducts(grouped)
print(multipliedElements.collect())

Matrix  A
        1.00        2.00        4.00
        2.00        3.00       10.00
       12.00       15.00      150.00
Matrix  B
        4.00        2.00        2.00      324.00       23.00
        1.00        3.00        3.00      333.00      423.00
       23.00       34.00       12.00       12.00        0.00

********** Representation for multipliedElements ************

[((0, 0), 4.0), ((0, 1), 2.0), ((0, 2), 2.0), ((0, 3), 324.0), ((0, 4), 23.0), ((1, 0), 8.0), ((1, 1), 4.0), ((1, 2), 4.0), ((1, 3), 648.0), ((1, 4), 46.0), ((2, 0), 48.0), ((2, 1), 24.0), ((2, 2), 24.0), ((2, 3), 3888.0), ((2, 4), 276.0), ((0, 0), 92.0), ((0, 1), 136.0), ((0, 2), 48.0), ((0, 3), 48.0), ((0, 4), 0.0), ((1, 0), 230.0), ((1, 1), 340.0), ((1, 2), 120.0), ((1, 3), 120.0), ((1, 4), 0.0), ((2, 0), 3450.0), ((2, 1), 5100.0), ((2, 2), 1800.0), ((2, 3), 1800.0), ((2, 4), 0.0), ((0, 0), 2.0), ((0, 1), 6.0), ((0, 2), 6.0), ((0, 3), 666.0), ((0, 4), 846.0), ((1, 0), 3.0), ((1, 1), 9.0), ((1, 2), 9.0), ((1, 3),

<p>
<font size="3">
Each element of the RDD $multipliedElements$ obtained above is $((i, k), a_{i, j} \cdot b_{j, k})$, 
$0 \leq i \leq n-1$, $0 \leq j \leq m-1$, $0 \leq k \leq p-1$. 
From Equation (1), we can see that each $c_{i, k}$ is obtained by summing up the products $a_{i, j} \cdot b_{j, k}$, $0 \leq j \leq m-1$.
<p>
Therefore, in order to obtain the matrix $C = A \times B$, the only thing that we need to do is to sum all 
values $a_{i, j} \cdot b_{j, k}$ associated with the same key $(i, k)$.
</p>
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the definition of the function $getResult()$ that:
    <ul>
      <li> Takes in the RDD $multipliedElements$.
      <li> Transforms the input RDD into one where each element is $((i, k), \sum\limits_{j=0}^{m-1} a_{i, j} \cdot b_{j, k})$
      <li> Returns the resulting RDD. 
    </ul>
    Execute the code.We finally obtain the matrix $C$.**</font>
<hr style=" border:none; height:2px;">
</p>



In [10]:
def getResult(multipliedElements):
      # Apply a transformation to the input RDD so that all values with the same key are summed.

    return multipliedElements.reduceByKey(lambda x, y: x + y)


nice("A", A)
nice("B", B)
C = getResult(multipliedElements)
nice("C", C)

############################################################## 
#YOU SHOULD OBTAIN THE FOLLOWING MATRIX C AS RESULT
# 98.00      144.00       56.00     1038.00      869.00
# 241.00      353.00      133.00     1767.00     1315.00
# 3513.00     5169.00     1869.00    10683.00     6621.00
##############################################################


Matrix  A
        1.00        2.00        4.00
        2.00        3.00       10.00
       12.00       15.00      150.00
Matrix  B
        4.00        2.00        2.00      324.00       23.00
        1.00        3.00        3.00      333.00      423.00
       23.00       34.00       12.00       12.00        0.00
Matrix  C
       98.00      144.00       56.00     1038.00      869.00
      241.00      353.00      133.00     1767.00     1315.00
     3513.00     5169.00     1869.00    10683.00     6621.00


<p>
<font size="3">
We now wrap every function that we implemented above in only one function $multiply()$ that we can use any time we need to multiply two matrices.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the code below to define the function $multiply()$**</font>
<hr style=" border:none; height:2px;">
</p>

In [11]:
def multiply(A, B):
  
    if shape(A)[1] != shape(B)[0]:
        sys.exit("The number of colums of A must equal the number of rows of B")
    A = transformA(A)
    B = transformB(B)
    C = merge(A, B)
    C = group(C)
    C = groupProducts(C)
    C = getResult(C)
    return C


## Exercise 4. PageRank

<p>
<font size="3">
**PageRank** is the algorithm used by the Google search engine to assign each Web page a numeric score (in short, _PR-score_) representing its importance. Google uses the PR-score to rank the Web pages returned in response to a search; the most important pages come first.

<p>    
The PR-score of a Web page $p$ depends on the number 
of Web pages linking to $p$ and on their PR-score. 
Stated otherwise, a Web page that is linked by many important Web pages is itself considered as important.
</p>
</font>
</p>

<p>
<font size="3">
PageRank is an iterative algorithm. At the beginning, all the $n$ Web pages have the same PR-score ($1/n$). 
At each step, the PR-score of each Web page $p$ is modified based on the PR-score of the pages linking to $p$. 
The algorithm stops when the PR-scores of all pages converge (i.e., they don't change anymore from one iteration to the next).

<p>
We're now going to implement the simplified version of PageRank and we'll see why this doesn't always work.
</p>
</font>
</p>   
   
<p>
<font size="3">
PageRank simulates the way a random surfer would browse the Web by 
randomly choosing the links to follow at any given page.
Referring to Figure 1, 
a random web surfer at page $A$ would choose to go 
either to page $B$, $C$ or $D$ with probability $1/3$, because 
there are three links leaving $A$. 
</font>
</p>

<p>
<font size="3">
There is no way a random surfer will visit $D$ from $C$ because the latter 
does not link to the former.
Similarly, a random surfer at $B$ would go to 
$A$ or $D$ with probability $1/2$, 
because $B$ has two outgoing links.
</font>
</p>

<p>
<font size="3">
By iterating many times these _random walks_
across the graph, PageRank simulates the behavior of multiple random 
surfers; the pages that receive a larger  number of visits are considered
more important than the ones that are visited only rarely.
</font>
</p>

<p>
<font size="3">
These transition probabilities at each node can be described 
in a $n\times n$ _transition matrix_ $M$, 
where $n$ is the number of nodes
in the graph and the element $m_{i, j}$ is the 
probability that a random surfer moves from $j$ to $i$.
</font>
</p>

<p>
<font size="3">
Assuming that the nodes are in alphabetical 
order (first row/column correspond to $A$, second row/column correspond to $B$ and so on), 
the transition matrix corresponding to the graph represented in 
Figure 1 is as follows:

<center>    
$M = 
\begin{bmatrix}
    0 & 1/2 & 1 & 0  \\
    1/3 & 0 & 0 & 1/2  \\
    1/3 & 0 & 0 & 1/2  \\
    1/3 & 1/2 & 0 & 0  
\end{bmatrix}$
</center>
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Complete the code below to load the matrix $M$ from the file matrix-m.txt_ and display the matrix calling the function $nice()$.**</font>
<hr style=" border:none; height:2px;">
</p>



In [12]:
# Get the matrix from file.
M = loadMatrix("./data/matrix-m.txt")
nice("M", M)



Matrix  M
        0.00        0.50        1.00        0.00
        0.33        0.00        0.00        0.50
        0.33        0.00        0.00        0.50
        0.33        0.50        0.00        0.00


<p>
<font size="3">    
When a random surfer starts her walk, she can be anywhere in the graph. 
If we have no reason to believe that she would be more likely to choose one 
page over another one as her starting point, we can say that the initial 
probability of the surfer of being at a certain page is $1/n$. 
</font>
</p>

<p>
<font size="3">    
The probability distribution of the position of the surfer can be described 
by a column vector $\pmb{v^{(0)}}$ with $n$ elements, which in our example looks like as 
follows:
<center>
$
\pmb{v^{(0)}} = 
\begin{bmatrix}
    1/4  \\
    1/4 \\
    1/4 \\
    1/4 
\end{bmatrix}
$
</center>
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Implement the function $initialization()$ that:
    <ul>
     <li> Takes in a matrix $M$.
     <li> Returns the vector $v_0$. $v_0$ must be an RDD, in the same way as the matrix $M$.
    </ul>
    Execute the code.**</font>
<hr style=" border:none; height:2px;">
</p>

In [13]:
'''
Initialization of the vector v0
'''
def initialize(M):    
    # Get the number n of rows of the input matrix
    (n, m) = shape(M)
    # Create the column vector having n elements equal to 1/n
    # The vector must be represented in the same way as the matrix M.
    # First, create a list L in Python where each element is ((i, 0), 1/n) for i=0, ..., n-1
    L = [((i, 0), 1./n) for i in range(n)]
    
    # Then tranforms this list into an RDD (remember the transformation parallelize...)
    L = sc.parallelize(L)
    
    return L
    

v0 = initialize(M)
nice("v0", v0)

Matrix  v0
        0.25
        0.25
        0.25
        0.25


<p>
<font size="3">
We now want to compute the vector $\pmb{v^{(1)}}$
that gives 
the distribution probability of the position of the surfer after 
one iteration.
The probability $\pmb{v^{(1)}}_i$
that the surfer will be at node $i$ after the
first iteration is expressed as follows:
<p>
<center>
$\pmb{v^{(1)}}_i = \sum \limits_{j=1}^n m_{i, j} \pmb{v^{(0)}}_j$
</center>
</p>
where $m_{i,j}$ is the probability that the surfer 
moves from node $j$ to node $i$ and 
$\pmb{v^{(0)}}_j$ is the 
probability that the surfer is at node $j$ at the iteration 0.
</font>
</p>

<p>
<font size="3">
Therefore, $\pmb{v^{(1)}}$ can be obtained by multiplying $M$ with $\pmb{v^{(0)}}$ :
<p>
<center>
$\pmb{v^{(1)}} = M \cdot \pmb{v^{(0)}}$
</center>
</p>
</font>
</p>


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Implement the function $performStep()$ that:
    <ul>
     <li> Takes in a matrix $M$ and a vector $v$
     <li> Returns the vector $M \cdot v$. 
    </ul>
    Execute the code.**</font>
<hr style=" border:none; height:2px;">
</p>


In [14]:
'''
Performs a step of the PageRank algorithm
This function returns a vector obtained by multiplying 
matrix M with vector v.
'''
def performStep(M, v):    
    # Multiply M and v. We already defined the function multiply() above to multiply two matrices.
    # Use it :)
    R = multiply(M, v)
    
    return R


v1= performStep(M, v0)
nice("v1", v1)

##############################################################
#
# YOU SHOULD OBTAIN THE FOLLOWING VALUES:
#  0.38
#  0.21
#  0.21
#  0.21
#
##############################################################

Matrix  v1
        0.38
        0.21
        0.21
        0.21


<p>
<font size="3">
If we want to obtain the 
probability distribution $\pmb{v^{(i)}}$ after
$i$ iterations, we compute:
<p>
<center>
$\pmb{v^{(i)}} = M \cdot \pmb{v^{(i-1)}}$
</center>
</p>
</font>
</p>

<p>
<font size="3">
The algorithm stops at the iteration $k$ where $\lvert \pmb{v^{(k-1)}} - \pmb{v^{(k)}} \rvert < \epsilon$, $\epsilon$ being an arbitrarily small constant.
</font>
</p>

<p>
<font size="3">
We now wrap up all the functions that we defined, to obtain an implementation of PageRank.
The code below defines a function $SimplifiedPageRank()$ that takes in a matrix, initializes the vector $v_0$
and calls a function $SimplifiedPageRankStep()$ that recursively performs a step until convergence.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Implement the function $converge()$ that:
    <ul>
     <li> Takes in two vectors $v_{prev}$ and $v_{next}$.
     <li> Returns whether $\lvert v_{prev} - v_{next} \rvert < \epsilon$
    </ul>
    Follow the comments in the code for the implementation. Execute the code.**</font>
<hr style=" border:none; height:2px;">
</p>



In [15]:
# Variable eps
eps = 0.001

'''
Returns true if |v_prev - v_next| < eps
'''
def converge(v_prev, v_next):    
    # Put together v_prev and v_next
    v = v_prev.union(v_next)
    
    # Each element of the RDD v is a key-value pair ((i, 0), e), where e is an element of 
    # either the vector v_prev or v_next, for i = 0, .., n-1
    # If you don't believe it, print it :)
    # We now want to obtain a new RDD from v, such that for any pair ((i, 0), e1), ((i, 0), e2) 
    # of elements of v, we obtain an element ((i, 0), |e1 - e2|).
    # In practice, the new RDD represents the vector |v_prev-v_next|.
    # Which transformation are you going to apply on v?
    v = v.reduceByKey(lambda x, y : abs(x - y)) 
    
    # Now we want to obtain a new RDD by keeping only the elements from v that are greater than eps.
    # Which transformation are you going to apply on v?
    v = v.filter(lambda kv : kv[1] > eps) 
    
    # Finally counts the number of elements left in v. Which actions are you going to apply on v?
    c = v.count()

    # If no element is greater than eps, then the algorithm converged.
    return c == 0
    

    
'''
One iteration of PageRank.
Calls recursively itself until convergence.
'''
def SimplifiedPageRankIteration(M, v):
    v_next = performStep(M, v)
    if converge(v, v_next):
        return v_next
    else:
        return SimplifiedPageRankIteration(M, v_next)

'''
Initializes the vector v0 and starts the iterations.
'''
def SimplifiedPageRank(M):
    (n, m) = shape(M)
    v0 = sc.parallelize([((i, 0), 1./n) for i in range(m)])
    return SimplifiedPageRankIteration(M, v0)

M = loadMatrix("./data/matrix-m.txt")
nice("M", M)
pr = SimplifiedPageRank(M)
nice("PageRank for M", pr)
############################################################## 
# YOU SHOULD OBTAIN THE FOLLOWING VECTOR FOR M AS RESULT
# 0.33
# 0.22
# 0.22
# 0.22
##############################################################


M1 = loadMatrix("./data/matrix-m1.txt")
nice("M1", M1)
pr = SimplifiedPageRank(M1)
nice("Page Rank for M1", pr)


############################################################## 
#YOU SHOULD OBTAIN THE FOLLOWING VECTOR PAGE_RANK AS RESULT
# 0.22
# 0.44
# 0.33
##############################################################
    

Matrix  M
        0.00        0.50        1.00        0.00
        0.33        0.00        0.00        0.50
        0.33        0.00        0.00        0.50
        0.33        0.50        0.00        0.00
Matrix  PageRank for M
        0.33
        0.22
        0.22
        0.22
Matrix  M1
        0.00        0.50        0.00
        0.50        0.00        1.00
        0.50        0.50        0.00
Matrix  Page Rank for M1
        0.22
        0.44
        0.33


# B. Text processing




## B.1. Support functions 


<p align="justify">
<font size="3">
Some support functions are provided. Read carefully the signatures of the fuctions.
<ul>
<li> $remove\_non\_letters(word)$
<li> $load\_stopwords(stopwords\_file)$
<li> $preprocess(text, stopwords)$
<li> $word\_count(words)$
</ul>
</font>
</p>


In [16]:
import re
# Regular expression for removing all non-letter characters in the file.
regex = re.compile('[^a-zA-Z ]')




'''
Removes any non-letter character from the given word.

INPUT:
        word: A word

OUTPUT:
        the input word without the non-letter characters.

'''
def remove_non_letters(word):
    return regex.sub('', word)


'''
INPUT: 
        stopwords_file: name of the file containing the stopwords.
OUTPUT:
        a Python list with the stopwords read from the file.
'''
def load_stopwords(stopwords_file):
    stopwords = []
    with open(stopwords_file) as file:
        for sw in file:
            stopwords.append(sw.strip())
    return stopwords


'''
INPUT: 
        text: RDD where each element is a line of the input text file.
        stopwords: Python list containing the stopwords.
OUTPUT: 
        RDD where each element is a word from the input text file.
'''
def preprocess(text, stopwords) :
  words = text.flatMap(lambda line: line.split(" ")).map(lambda word: remove_non_letters(word)).filter(lambda word: len(word) > 0).map(lambda word: word.lower()).filter(lambda word: word not in stopwords)
  return words

'''
Returns how many times a word appears in a RDD 
INPUT:
        words: RDD, where each element is word from the input text file (preprocessing already done!).
OUTPUT:
        RDD, where each element is (w, occ), w is a word and occ the number of occurrences of w.
        The RDD is sorted by value in decreasing order.
'''

def word_count(words):    
    occs = words.map(lambda word: (word, 1))\
                .reduceByKey(lambda x, y: x+y)\
                .sortBy(lambda f: f[1], ascending=False)
    return occs

# Storing in stopwords the list of the stopwords that is provided
stopwords = load_stopwords("./data/stopwords.txt")


##  Exercise 5. Inverted Index

<hr style="border:solid 2px;">


<p align="justify">
<font size="3">
In the folder _./data/bbc_ you'll find a collection of 50 documents from the BBC news website corresponding to stories in five topics. The five topics are:
<ul>
<li> _business_ 
<li>_entertainment_
<li> _politics_
<li> _sport_ 
<li> _tech_
</ul>

In the directory, the stories are text files (named: $\_001.txt\_$, $\_002.txt\_$, ...) organized into five directories, one for topic.
</font>
</p>

<p align="justify">
<font size="3">
In this exercise, we want to create an **inverted index**. An inverted index is an essential component of a search engine. In fact, given any word, the inverted index allows the search engine to quickly retrieve all documents containing that word.

An inverted index associates each word (you can find in the files) to the list of the names files the word occurs in.

More precisely, for each word, the inverted index will have a list of the names  of the files (path relative to the folder _./data_) that contain the word. 


(family, \[./data/bbc/tech/006.txt, ./data/bbc/entertainment/003.txt, ./data/bbc/entertainment/005.txt, ...\]
</font>
</p>

<p align="justify">
<font size="3">
The function $inverted\_index$ has the following input and output:
<ul>
    <li> **Input.** A RDD $files$, where each element is $(f, content)$, $f$ being the name of a text file in the collection and $content$ being the content of that file; 
a Python list $stopwords$, containing the most common English stopwords.
    <li> **Output.** A RDD, where each element is $(w, L)$, $w$ is a word and $L$ is the list of the names of the files containing $w$. The list must not contain duplicate file names.
</ul>
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>**Write the code of the function $inverted\_index()$. The function must apply a sequence of RDD transformations to:**
<ol>
  <li> split the content of each file into its constituent words.
  <li> lowercase each word.
  <li> remove the non-letter characters from each word (you can use the function $remove\_non\_letters$ defined in Exercise 1).
  <li> remove empty words.
  <li> remove the stopwords.
  <li> remove duplicate words.
</ol>
</font>
</p>
<hr style="border:solid 2px;">

In [17]:
'''
INPUT:
        files: RDD, each element is (f, content), where f is the name of a file in the collection and content is 
                its content.
        stopwords: a Python list containing the stopwords.

OUTPUT:y

        a RDD, each element is (w, L), where w is a word and L is the list of the names of the files containing
        w (without repetition).

'''

def inverted_index(files, stopwords):
    '''############## WRITE YOUR CODE HERE ##############'''

    #Split the content of each file into its constituent words and lowercase each word.
    line_split = files.flatMap(lambda f:[(f[0], word) for word in f[1].lower().split()])       
    #remove the non-letter characters from each word (you can use the function  𝑟𝑒𝑚𝑜𝑣𝑒_𝑛𝑜𝑛_𝑙𝑒𝑡𝑡𝑒𝑟𝑠  defined in Exercise 1).
    no_non_letters = line_split. map(lambda f: (f[0], remove_non_letters(f[1])))
    #remove empty words
    no_empty = no_non_letters.filter(lambda f: len(f[1]) > 0)
    #remove the stopwords
    no_stop_words = no_empty.filter(lambda f: f[1] not in stopwords)
    #remove duplicate words.
    duplicates_removed = no_stop_words.distinct()
    #produce the inverted index dictionary
    output = duplicates_removed.map(lambda f: (f[1], f[0])).groupByKey()
    return output

    '''############## END OF THE EXERCISE ##############'''

'''
INPUT:
        iindex: RDD containing the inverted index, as returned by the function inverted_index.
        word: a word.

OUTPUT:
        prints the list of the files contain the given word.
'''
def lookup(iindex, word):
    ld = iindex.sortByKey().lookup(word)
    if len(ld) > 0:
        print("The following documents contain the word '",word,"'")
        for d in sorted(ld[0]):
            print(os.path.relpath(d[5:], os.getcwd()))
    else:
        print("No documents contain the word '",word,"'")

####################   GOOD TO KNOW  ####################
# The Spark function wholeTextFiles loads into a RDD the content of the text files contained
# in the given directory.
# Each item of the RDD is a pair (f, content), where f is the name of a file and content is the content
# of the file.
#######################################################
file_collection = sc.wholeTextFiles("./data/bbc/*")        
iindex = inverted_index(file_collection, stopwords)
#iindex.lookup()
lookup(iindex, "family")

################# EXPECTED OUTPUT #################
#
# data/bbc/entertainment/002.txt
# data/bbc/entertainment/003.txt
# data/bbc/entertainment/005.txt
# data/bbc/politics/001.txt
# data/bbc/sport/004.txt
# data/bbc/tech/004.txt
# data/bbc/tech/006.txt
#
###################################################

The following documents contain the word ' family '
data/bbc/entertainment/002.txt
data/bbc/entertainment/003.txt
data/bbc/entertainment/005.txt
data/bbc/politics/001.txt
data/bbc/sport/004.txt
data/bbc/tech/004.txt
data/bbc/tech/006.txt


##  Exercise 6 Co-occurrence Matrix


<hr style="border:solid 2px;">


<p align="justify">
<font size="3">
Given the BBC collection, we want to calculate the **co-occurrence matrix** $M$, such that $M[w_1][w_2]$ is the number of documents in which two words $w_1$ and $w_2$ appear together.
</font>
</p>

<p align="justify">
<font size="3">
The function $co\_occurrence\_matrix()$ has the following input and output:
<ul>
 <li> **Input.** A RDD $files$ and a Python list $stopwords$, as in the previous exercise.
 <li> **Output.** A RDD, where each element is $((w_1, w_2), occ)$, where $w_1$ and $w_2$ are words and $occ$ is the number of files in which the two words appear together.
</ul>
As in the case of the function $inverted\_index()$, words must be lowercases, non-letter characters, empty words and stopwords should be removed.
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>**Write the code of the function $co\_occurrence\_matrix()$. You can draw inspiration from the MapReduce algorithms that we discussed in class. Also, you can use the already implemented function $create\_pairs()$ to generate all the possible pairs from a list of words. The function assumes that the words in the input list are sorted lexicographically.**
<br>
</font>
</p>

<hr style="border:solid 2px;">

In [48]:
'''
INPUT:
        words: Python list containing words. IMPORTANT: the function assumes that the 
        list is sorted in lexicographic order.
OUTPUT:
        Python list containing all possible pairs from the given list.
'''
def create_pairs(words):
    n = len(words)
    output = []
    for i in range(0, n):
        for j in range(i+1, n):
            output.append((words[i], words[j]))
    return output

'''
INPUT:
        files: RDD, each item is (f, content), where f is the name of a file and content is the text of the file.
        stopwords: A RDD, each item is ((w1, w2), occ), where w1 and w2 are words and occ is the number of
                    files in which w1 and w2 appear together.
'''
def co_occurrence_matrix(files, stopwords):
    '''############## WRITE YOUR CODE HERE ##############'''
    
    output = files.flatMap(lambda f:[(f[0], word) for word in re.compile("\s").split(f[1].lower())])\
                .map(lambda f: (f[0], remove_non_letters(f[1])))\
                .filter(lambda f: len(f[1]) > 0)\
                .filter(lambda f: f[1] not in stopwords)\
                .distinct()\
                .groupByKey()\
                .map(lambda f: (f[0], sorted(f[1])))\
                .map(lambda f: (f[0], create_pairs(f[1])))\
                .flatMap(lambda f: [(word_pair, 1) for word_pair in f[1]])\
                .reduceByKey(lambda x, y: x+y)
    return output


file_collection = sc.wholeTextFiles("./data/bbc/*")
output = co_occurrence_matrix(file_collection, stopwords)    
#output.count()
output.takeOrdered(10, key = lambda x: -x[1])

################# EXPECTED OUTPUT #################
#
#[(('also', 'said'), 24),
# (('said', 'world'), 20),
# (('new', 'said'), 19),
# (('said', 'year'), 17),
# (('also', 'world'), 17),
# (('one', 'said'), 16),
# (('said', 'set'), 15),
# (('said', 'time'), 15),
# (('last', 'said'), 15),
# (('back', 'said'), 14)]
#
###################################################

[(('also', 'said'), 24),
 (('said', 'world'), 20),
 (('new', 'said'), 19),
 (('said', 'year'), 17),
 (('also', 'world'), 17),
 (('one', 'said'), 16),
 (('said', 'set'), 15),
 (('said', 'time'), 15),
 (('last', 'said'), 15),
 (('back', 'said'), 14)]

In [21]:
len(sorted(set(iindex.lookup('said')[0]).intersection(set(iindex.lookup('world')[0]))))

20

<hr style=" border:solid 2px;">

##  Exercise 7 - OPTIONAL - enjoy with what you just wrote

<p align="justify">
<font size="3">
We want to code a function $term\_freq$ that computes the frequency of each word in a 
text document. 
More precisely, given a document $d$ and a word $w$ in that document, we want to 
compute its frequency $tf(w, d)$, as follows:
    
<p>    
$$ tf(w, d) = \frac{f_{w, d}}{\sum\limits_{w^\prime \in d} f_{w^\prime, d}}$$
</p>

where $f_{w, d}$ is the number of occurrences of word $w$ in $d$.
</font>
</p>

<p>
<font size="3">
The function $term\_freq$ has the following input and output:
<ul>
<li> **Input.** A RDD $words$, where each element is a word in a text document $d$ (pre-processing already done).
<li> **Output.** A RDD, where each element is a key-value pair $(w, tf(w, d))$.
</ul>
</font>
</p>
<p align="justify">
<font size="3" color='#91053d'>**Write the code of the function $term\_freq$. You can take advantage of the 
    function $word\_count$.**
</font>
</p>

<hr style=" border:solid 2px;">

In [19]:
def term_freq(words):
    '''############## WRITE YOUR CODE HERE ##############'''
    
    raw_counts = word_count(words)
    sum_counts = raw_counts.values().reduce(lambda x, y: x+y)
    raw_counts = raw_counts.map(lambda x: (x[0], x[1]/sum_counts))
    return raw_counts

    '''############## END OF THE EXERCISE ##############'''
 
text = sc.textFile('./data/bbc/politics/001.txt')
words = preprocess(text, stopwords)
tf = term_freq(words)
tf.take(5)

################# EXPECTED OUTPUT #################
#
#[('pay', 0.045081967213114756),
# ('maternity', 0.036885245901639344),
# ('months', 0.036885245901639344),
# ('said', 0.036885245901639344),
# ('plans', 0.020491803278688523)]
#
###################################################

[('pay', 0.045081967213114756),
 ('maternity', 0.036885245901639344),
 ('months', 0.036885245901639344),
 ('said', 0.036885245901639344),
 ('plans', 0.020491803278688523)]